# CausalPilot: Advanced Features 🛡️

In this notebook, we'll explore the advanced features of CausalPilot that make your analysis robust and professional:

1. **Diagnostics**: Checking Covariate Balance (Love Plots).
2. **Refutation**: Stress-testing your model with Placebo Treatments.
3. **X-Learner**: Handling unbalanced treatment groups.
4. **Instrumental Variables (IV)**: Handling unobserved confounding.


In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import causalpilot as cp
from causalpilot.core import CausalModel, Refutation
from causalpilot.visualization.diagnostics import plot_covariate_balance
from causalpilot.datasets import load_ihdp

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Diagnostics: Covariate Balance

Before estimating effects, we must ensure that the treatment and control groups are comparable. We use **Standardized Mean Differences (SMD)** to check this.

In [ ]:
ihdp_data = load_ihdp()
covariates = [col for col in ihdp_data.columns if col not in ['treatment', 'outcome']]

# Plot Love Plot (Covariate Balance)
plot_covariate_balance(ihdp_data, treatment='treatment', covariates=covariates)

## 2. Refutation: Placebo Treatment

How do we know our result isn't just noise? We can replace the treatment with a random variable. The effect *should* go to zero. If it doesn't, our model is broken.

In [ ]:
# Initialize model
model = CausalModel(ihdp_data, 'treatment', 'outcome')
model.identify_effect()

# Run Refutation
refuter = Refutation(model)
result = refuter.placebo_treatment_refutation(n_simulations=5)

print(f"Original Effect: {result['original_effect']:.3f}")
print(f"Placebo Effect (should be ~0): {result['new_effect']:.3f}")
print(f"Passed: {result['passed']}")

## 3. X-Learner for Unbalanced Data

When one group is much smaller than the other (e.g., only 10% treated), standard learners fail. The **X-Learner** is designed for this.

In [ ]:
from causalpilot.inference import XLearner

xl = XLearner()
xl.fit(ihdp_data[covariates], ihdp_data['treatment'], ihdp_data['outcome'])
ate_xl = xl.estimate_effect()

print(f"ATE (X-Learner): {ate_xl:.3f}")

## 4. Instrumental Variables (IV)

What if we have unobserved confounders? We can use an **Instrument** (Z) that affects treatment but not the outcome directly.

In [ ]:
from causalpilot.inference import IV2SLS

# Simulate IV data
n = 1000
Z = np.random.binomial(1, 0.5, n)  # Instrument
U = np.random.normal(0, 1, n)      # Unobserved Confounder
X = np.random.normal(0, 1, (n, 1)) # Observed Covariate
T = (0.8*Z + 0.5*X[:,0] + U > 0.5).astype(int) # Treatment (confounded by U)
Y = 2.0*T + 1.0*X[:,0] + U + np.random.normal(0, 0.1, n) # Outcome (True Effect = 2.0)

# Estimate with IV
iv = IV2SLS()
iv.fit(X, T, Y, Z=Z)
ate_iv = iv.estimate_effect()

print(f"True Effect: 2.0")
print(f"Estimated Effect (IV): {ate_iv:.3f}")